In [20]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio

os.chdir('/sharedfolder/sida_classifier/')

speaker_1_label = 'Barack Obama'

In [21]:
def load_features(dir_path):
    features = []
    for filename in [item for item in os.listdir(dir_path) if 'Weekly' in item]:
        with open(os.path.join(dir_path, filename)) as fi:
            csv_reader = csv.reader(fi)
            for row in csv_reader:
                features.append([float(item) for item in row])
    return features

speaker_1_features = load_features('/sharedfolder/sida_classifier/_classes_Obama_training_clips_all/Barack_Obama/_vowel_mfccs_and_deltas')
print(len(speaker_1_features))

9090


In [22]:
## Load saved features

def load_features(dir_path):
    features = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename)) as fi:
            csv_reader = csv.reader(fi)
            for row in csv_reader:
                features.append([float(item) for item in row])
    return features

#speaker_1_features = load_features('/sharedfolder/sida_classifier/_classes_Obama_training_clips_all/Barack_Obama/_vowel_mfccs_and_deltas')
#print(len(speaker_1_features))

aapb_ubm_male_features = load_features('/sharedfolder/sida_classifier/AAPB_male_vowel_mfccs_and_deltas')
print(len(aapb_ubm_male_features))

aapb_ubm_female_features = load_features('/sharedfolder/sida_classifier/AAPB_female_vowel_mfccs_and_deltas')
print(len(aapb_ubm_female_features))

11531
4382


In [23]:
## Printing MFCCs and deltas for a single frame

print(random.choice(speaker_1_features))

[-31.570021010058156, 53.19964960524807, -28.480197531102924, 22.79863102988201, -7.504472796130757, -13.87715614587367, 7.707559153126783, -0.3399791834840653, 3.10885365573549, -9.907691154139238, 11.81646759280599, -9.320023270006864, -3.6086779400261513, -19.674495362834467, 6.853691549479249, -7.797116294891144, -3.3148994473777327, 1.8392980077153793, -1.633601280358091, 2.5009395696662673, 0.3992897351999418, 1.6736759962222547, 2.431992302298922, 3.454357103296964, -2.740537870714971, 3.394858308480474, -4.075983445414886, 0.9509733807967413, -1.4379045271160305, -0.9991704982142343, 0.4972250828040799, -0.21336966423752973, 0.47707185702905697, 0.09361773690314319, 0.3420342807529463, 0.6572837252854712, 0.645936263877306, -0.513065206799681]


In [24]:
## Combining feature sets

speaker_1_features = speaker_1_features
ubm_features = aapb_ubm_male_features + aapb_ubm_female_features

print(len(speaker_1_features))
print(len(ubm_features))

9090
15913


In [25]:
## Training and multi-layer perceptron model with 9/10 of training data and evaluating performance on remaining 1/10

os.chdir('/sharedfolder/sida_classifier/')

from sklearn.neural_network import MLPClassifier

X = speaker_1_features[:-len(speaker_1_features)/10] + ubm_features[:-len(ubm_features)/10]
y = [1]*len(speaker_1_features[:-len(speaker_1_features)/10]) + [0]*len(ubm_features[:-len(ubm_features)/10])

X_test = speaker_1_features[-len(speaker_1_features)/10:] + ubm_features[-len(ubm_features)/10:]
y_test = [1]*len(speaker_1_features[-len(speaker_1_features)/10:]) + [0]*len(ubm_features[-len(ubm_features)/10:])

classifier = MLPClassifier().fit(X, y)

print(classifier.score(X_test,y_test))

0.882447021192


In [26]:
## Training and saving an MLP model with all training data

from sklearn.neural_network import MLPClassifier

X = speaker_1_features + ubm_features
y = [1]*len(speaker_1_features) + [0]*len(ubm_features)

classifier = MLPClassifier().fit(X, y)

trained_model_filename = speaker_1_label.replace(' ', '_') + '_vowels_mlpc_4096_weekly_addresses.pkl'
print(trained_model_filename)

## Saving trained model
joblib.dump(classifier, trained_model_filename)
classifier = joblib.load(trained_model_filename)

Barack_Obama_vowels_mlpc_4096_weekly_addresses.pkl


In [10]:
##############################################
#### Start here to load pre-trained model ####
##############################################

#trained_model_filename = speaker_1_label.replace(' ', '_') + '_vowels_mlpc_2048.pkl'

#os.chdir('/sharedfolder/sida_classifier/')
#classifier = joblib.load(trained_model_filename)

In [11]:
%%capture
## Download unseen audio and split into 3-second WAV clips for testing

os.chdir('/sharedfolder/sida_classifier/')

try: os.mkdir('test_clips/')
except: pass

os.chdir('/sharedfolder/sida_classifier/test_clips/')

mp3_url = "https://www.podtrac.com/pts/redirect.mp3/cdn.pri.org/sites/default/files/episode-audio/01202014full.mp3"

mp3_filename = os.path.basename(mp3_url)

wav_filename = mp3_filename[:-4]+'.wav'

subprocess.call(['wget', '-N', mp3_url])

subprocess.call(['ffmpeg', '-i', mp3_filename, wav_filename])

subprocess.call(['ffmpeg', '-i', wav_filename, '-f', 'segment', '-segment_time', '3',  wav_filename[:-4] + '_3_sec_%04d.wav'])


In [44]:
## Classifying short clips
#### Repeat this cell several times to help choose a classifier threshold value.

os.chdir('/sharedfolder/sida_classifier/test_clips/')

wav_pathname = os.path.abspath(random.choice([item for item in os.listdir('./') if '3_sec' in item]))

test_features = attk.get_mfccs_and_deltas(wav_pathname)

print(wav_pathname)

results = classifier.predict(test_features)  ## Predicting new observation

print(results)

vowel_results=[]

vowel_bools = attk.get_vowel_segments(wav_pathname)

for i in range(len(results)):
    try:
        if vowel_bools[i]==True:
            vowel_results.append(results[i])
    except:
        pass

display(Audio(wav_pathname))

print("All samples: "+str(np.mean(results)))
print("Vowels only: "+str(np.mean(vowel_results)))

/sharedfolder/sida_classifier/test_clips/WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA_3_sec_0542.wav
[1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0]


All samples: 0.575757575758
Vowels only: 0.458333333333


In [45]:
## Function that classifies vowel segments only and returns 
## average output for the full clip

def classify_clip(clip_pathname):
    mfccs = attk.get_mfccs_and_deltas(clip_pathname)
    results = classifier.predict(mfccs)  ## Predicting new observation
    vowel_results=[]
    vowel_bools = attk.get_vowel_segments(clip_pathname)

    for i in range(len(results)):
        if vowel_bools[i]==True:
            vowel_results.append(results[i])

    return np.mean(vowel_results) ## Vowels only

In [ ]:
%%capture
## Classifying a long audio file

## Writing classification output to CSV

classifier_threshold = 0.0      # Classifier values below this threshold will be discarded


resolution_secs = 3.0

media_path = "/sharedfolder/sida_classifier/test_clips/WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.wav"


os.chdir('/sharedfolder/sida_classifier')

csv_path = media_path[:-4]+'_mlpc4096_labels_weekly_address_model_'+str(resolution_secs)+'s.csv'

counter=0


import timeit
tic=timeit.default_timer()

snd = AudioFileClip.AudioFileClip(media_path)

classifications = []

for i in range(int(attk.duration(media_path)/resolution_secs)):
    try:
        snd.subclip(i * resolution_secs , (i * resolution_secs) + resolution_secs).write_audiofile('/tmp/temp_clip.wav')
        value = classify_clip('/tmp/temp_clip.wav')
        
        with open(csv_path,'a') as fo:
            duration = resolution_secs

            if value >= classifier_threshold:
                start = i * resolution_secs
                fo.write(str(start) + ',' + str(duration) +','+ str(value) + ',' + speaker_1_label + '\n')
        
    except:
        classifications.append(0.0)
        print("Error: " + str(i))

In [ ]:
print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
## Writing classification output to CSV

classifier_threshold = 0.0      # Classifier values below this threshold will be discarded

os.chdir('/sharedfolder/sida_classifier')

csv_path = media_path[:-4]+'_mlpc2048_labels_'+str(resolution_secs)+'s.csv'

counter=0

with open(csv_path,'w') as fo:
    duration = resolution_secs
    for value in classifications:
        if value >= classifier_threshold:
            start = counter * resolution_secs
            fo.write(str(start) + ',' + str(duration) +','+ str(value) + ',' + speaker_1_label + '\n')
        counter+=1